<a href="https://colab.research.google.com/github/Manav29/Movie-Reviews-Classification/blob/master/Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from nltk.corpus import movie_reviews

In [4]:
movie_reviews.categories()

['neg', 'pos']

In [0]:
movie_reviews.fileids();     #1st 1000 are Neg, Last 1000 are Pos

In [6]:
movie_reviews.words(movie_reviews.fileids()[5])

['capsule', ':', 'in', '2176', 'on', 'the', 'planet', ...]

In [0]:
documents = []
for category in movie_reviews.categories():
    for fileId in movie_reviews.fileids(category):
        documents.append( (movie_reviews.words(fileId),category) )

In [8]:
#For Easy Train Test Split
import random
random.shuffle(documents)
documents[:5]

[(['jay', 'and', 'silent', 'bob', 'strike', 'back', ',', ...], 'pos'),
 (['robin', 'williams', 'is', 'a', 'comedic', 'genus', ...], 'neg'),
 (['as', 'with', 'his', 'other', 'stateside', 'releases', ...], 'pos'),
 (['wesley', 'snipes', 'is', 'a', 'master', 'of', ...], 'neg'),
 (['a', 'sci', 'fi', '/', 'comedy', 'starring', 'jack', ...], 'pos')]

In [0]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [0]:
from nltk.corpus import wordnet
from nltk import pos_tag

def get_simpler_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [0]:
from nltk.corpus import stopwords
stops = stopwords.words('english')

import string
punctuations = list(string.punctuation)         

stops+=punctuations

In [0]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w,get_simpler_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [27]:
#Clean for all 2000 reviews in the document
documents = [ (clean_review(document),category) for document,category in documents]
documents[0][0][:10],documents[0][-1]           #See Tail

(['jay',
  'silent',
  'bob',
  'strike',
  'back',
  'kevin',
  'smith',
  'swan',
  'song',
  'two'],
 'pos')

In [28]:
len(documents)

2000

In [0]:
training_documents = documents[:1500]
testing_documents = documents[1500:]

In [30]:
all_words = []
for i in training_documents:
    all_words.extend(i[0])

len(all_words)
#all_words

534696

In [33]:
import nltk
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)            #List of Tuples (word,freq). We only need the words
features = [i[0] for i in common]
features[:25]

['film',
 'movie',
 'one',
 'make',
 'like',
 'character',
 'get',
 'see',
 'go',
 'time',
 'well',
 'scene',
 'even',
 'good',
 'story',
 'take',
 'would',
 'much',
 'bad',
 'also',
 'come',
 'give',
 'life',
 'two',
 'way']

In [0]:
def get_features_dict(words):
    new_features = {}
    for w in features:
        new_features[w] = w in words        #Boolean
    return new_features

In [43]:
#(get_features_dict(training_documents[0][0]))
import itertools  
dict(itertools.islice((get_features_dict(training_documents[0][0])).items(), 25))

{'also': False,
 'bad': False,
 'character': True,
 'come': True,
 'even': True,
 'film': True,
 'get': True,
 'give': True,
 'go': True,
 'good': False,
 'life': False,
 'like': True,
 'make': True,
 'movie': True,
 'much': False,
 'one': True,
 'scene': False,
 'see': True,
 'story': True,
 'take': False,
 'time': True,
 'two': True,
 'way': False,
 'well': True,
 'would': True}

In [0]:
training_features = [(get_features_dict(doc),category) for doc,category in training_documents]
testing_features = [(get_features_dict(doc),category) for doc,category in testing_documents]

In [0]:
training_features

In [0]:
from nltk import NaiveBayesClassifier

In [0]:
classifier = NaiveBayesClassifier.train(training_features)

In [48]:
#nltk.classify.accuracy(classifier,training_features)
nltk.classify.accuracy(classifier,testing_features)

0.78

In [49]:
classifier.show_most_informative_features(15)

Most Informative Features
             outstanding = True              pos : neg    =     13.2 : 1.0
             wonderfully = True              pos : neg    =      9.3 : 1.0
                 idiotic = True              neg : pos    =      7.9 : 1.0
                   jolie = True              neg : pos    =      7.7 : 1.0
                 emperor = True              pos : neg    =      7.6 : 1.0
                   mulan = True              pos : neg    =      7.6 : 1.0
               ludicrous = True              neg : pos    =      7.6 : 1.0
              schumacher = True              neg : pos    =      7.0 : 1.0
                  seagal = True              neg : pos    =      7.0 : 1.0
            breathtaking = True              pos : neg    =      6.5 : 1.0
                 garbage = True              neg : pos    =      6.1 : 1.0
                   anger = True              pos : neg    =      6.1 : 1.0
                    lame = True              neg : pos    =      5.8 : 1.0

SVM

In [0]:
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier

In [0]:
svc = SVC()
sklearn_to_nltk_classifier = SklearnClassifier(svc)         #Now input in ( {},cat ) format

In [52]:
sklearn_to_nltk_classifier.train(training_features)

<SklearnClassifier(SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))>

In [53]:
nltk.classify.accuracy(sklearn_to_nltk_classifier,testing_features)

0.822

Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=5,n_estimators=100)

In [0]:
#from nltk.classify.scikitlearn import SklearnClassifier
sklearn_to_nltk_classifier1 = SklearnClassifier(rfc)   

In [56]:
sklearn_to_nltk_classifier1.train(training_features)

<SklearnClassifier(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False))>

In [57]:
#nltk.classify.accuracy(sklearn_to_nltk_classifier1,training_features)
nltk.classify.accuracy(sklearn_to_nltk_classifier1,testing_features)

0.802